## Part I: EDA for the training data set of drug reviews

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import datetime

In [2]:
url='data/drugsComTest_raw.tsv'
df=pd.read_csv(url, sep="\t", index_col=0)

In [3]:
df.head(3)

,drugName,condition,review,rating,date,usefulCount
163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3


In [4]:
df.shape

(53766, 6)

In [5]:
df.isna().sum()

drugName         0
condition      295
review           0
rating           0
date             0
usefulCount      0
dtype: int64

In [6]:
#remove samples with missing conditions
df_new=df.dropna(0, how='any')
df_new.shape

(53471, 6)

### 1. date 

  **Qs:**
  
<span style='color:Blue'>Is there any trend for year, month or weekday when people tend to write drug review online?</span>  

In [7]:
import warnings
warnings.filterwarnings("ignore")
df_new.columns

Index(['drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'], dtype='object')

In [8]:
import datetime
df_new.date=pd.to_datetime(df_new.date)
df_new.date[:5]

163740   2012-02-28
206473   2009-05-17
159672   2017-09-29
39293    2017-03-05
97768    2015-10-22
Name: date, dtype: datetime64[ns]

In [9]:
#we can check the time_series distribution for drugs 
df_new['year']=df_new.date.dt.year
df_new['month']=df_new.date.dt.month
df_new['weekday']=df_new.date.dt.weekday

In [10]:
df_new.columns

Index(['drugName', 'condition', 'review', 'rating', 'date', 'usefulCount',
       'year', 'month', 'weekday'],
      dtype='object')

In [11]:
df_new.year.value_counts()

2016    11739
2017     9216
2015     9009
2013     4065
2014     4040
2009     3969
2011     3710
2012     3335
2010     2759
2008     1629
Name: year, dtype: int64

In [12]:
df_new.month.value_counts()

8     4902
3     4691
1     4668
7     4642
10    4568
11    4531
4     4460
9     4384
5     4292
6     4171
12    4091
2     4071
Name: month, dtype: int64

In [13]:
df_new.weekday.value_counts()

2    8405
1    8296
3    8056
0    8004
4    7385
5    6674
6    6651
Name: weekday, dtype: int64

<span style='color:Blue'>The amount of reviews has been greatly increased since 2015. But people didn't show any obvious preference to write reviews on certain month or certain weekday. </span>

### 2. condition

   **Qs:**
     <span style='color:Blue'>How many related conditions were mentioned in this data set? 
     What's the top conditions with most reviews? Is there any time dependent trend for condition?</span>

In [14]:
conditions=df_new.condition.unique().tolist()
len(conditions)

708

In [15]:
def top_con(y):
    r=df_new[df_new.year==y].condition.value_counts().head(10)
    new=r.to_frame().reset_index()
    new.columns=['condition','count']
    year=pd.Series([y, y, y, y, y, y, y, y, y, y]).T
    new['year']=year
    return new

In [16]:
top_con(2015)

,condition,count,year
0,Birth Control,1904,2015
1,Depression,413,2015
2,Acne,354,2015
3,Obesity,353,2015
4,Anxiety,290,2015
5,Weight Loss,239,2015
6,Insomnia,231,2015
7,Emergency Contraception,211,2015
8,Bipolar Disorde,208,2015
9,Pain,178,2015


In [17]:
top_condition_year=pd.concat([top_con(y) for y in range(2008, 2018)], ignore_index=True)

In [18]:
#Check the change of top conditions during 2007 to 2017
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(top_condition_year).mark_bar().encode(
    x='count',
    y='condition'
).transform_filter(
    click
)

hist = alt.Chart(top_condition_year).mark_point().encode(
    y='year:N',
    color=alt.condition(click, 'year:N', alt.value('lightgray'), legend=None)
).add_selection(
    click
)

scatter | hist

alt.HConcatChart(...)

In [19]:
#top_4_condition
top_4_con=df_new.condition.value_counts().head(4)
top_4_con.index

Index(['Birth Control', 'Depression', 'Pain', 'Anxiety'], dtype='object')

<span style='color:blue'>There are totally 708 conditions in this data set. Among them, birth control, depression, pain and anxiety are the top 4 based on the total review counts between 2008 and 2017. In 2008, pain was the primary conditions in these reviews, but birth control and depression became the dominant ones from 2010 to 2017.</span>

### 3. drugName

   **Qs:**
 <span style='color:Blue'>How many drugs were included in this datset? Which drugs have most reviews? What are the top drugs with high ratings? Which drugs in the top condition categories have most reviews? Are there multiple drugs used for the same condition? Are there multiple conditions using the same drug?</span>
   
   #### 3.1 check drugs without differentiation of condition


In [20]:
drugs=df_new.drugName.unique().tolist()
len(drugs)

2635

In [21]:
#the top 20 drugs in this data set
drug_top_20_list=df_new.drugName.value_counts().head(20)
drug_top_20_list

Levonorgestrel                        1265
Etonogestrel                          1081
Ethinyl estradiol / norethindrone      869
Nexplanon                              736
Ethinyl estradiol / norgestimate       649
Ethinyl estradiol / levonorgestrel     591
Phentermine                            539
Sertraline                             506
Escitalopram                           452
Mirena                                 431
Implanon                               404
Gabapentin                             367
Bupropion                              343
Miconazole                             343
Venlafaxine                            320
Duloxetine                             320
Tramadol                               316
Clonazepam                             315
Citalopram                             312
Medroxyprogesterone                    302
Name: drugName, dtype: int64

In [22]:
drug_top_20=df_new[df_new.drugName.isin(drug_top_20_list.index)]
drug_top_20.shape

(10461, 9)

In [23]:
drug_top_20.groupby(['drugName','condition']).count().head(20)

review  rating  date  \
drugName   condition                                                
Bupropion  ADHD                                  14      14    14   
           Anxiety                               20      20    20   
           Bipolar Disorde                        9       9     9   
           Depression                           198     198   198   
           Major Depressive Disorde              28      28    28   
           Migraine Prevention                    1       1     1   
           Not Listed / Othe                      1       1     1   
           Obesity                                1       1     1   
           Panic Disorde                          3       3     3   
           Persistent Depressive Disorde          1       1     1   
           Premenstrual Dysphoric Disorde         1       1     1   
           Seasonal Affective Disorde             3       3     3   
           Sexual Dysfunction, SSRI Induced      11      11    11   
           Smoking Cessation                     52      52    52   
Citalopram Agitation                              1       1     1   
           Anxiety and Stress                   143     143   143   
           Bipolar Disorde                        1       1     1   
           Depression                           125     125   125   
           Generalized Anxiety Disorde           20      20    20   
           Hot Flashes                            1       1     1   

                                             usefulCount  year  month  weekday  
drugName   condition                                                            
Bupropion  ADHD                                       14    14     14       14  
           Anxiety                                    20    20     20       20  
           Bipolar Disorde                             9     9      9        9  
           Depression                                198   198    198      198  
           Major Depressive Disorde                   28    28     28       28  
           Migraine Prevention                         1     1      1        1  
           Not Listed / Othe                           1     1      1        1  
           Obesity                                     1     1      1        1  
           Panic Disorde                               3     3      3        3  
           Persistent Depressive Disorde               1     1      1        1  
           Premenstrual Dysphoric Disorde              1     1      1        1  
           Seasonal Affective Disorde                  3     3      3        3  
           Sexual Dysfunction, SSRI Induced           11    11     11       11  
           Smoking Cessation                          52    52     52       52  
Citalopram Agitation                                   1     1      1        1  
           Anxiety and Stress                        143   143    143      143  
           Bipolar Disorde                             1     1      1        1  
           Depression                                125   125    125      125  
           Generalized Anxiety Disorde                20    20     20       20  
           Hot Flashes                                 1     1      1        1

In [24]:
def top_drug(y):
    r=df_new[df_new.year==y].drugName.value_counts().head(10)
    new=r.to_frame().reset_index()
    new.columns=['drugName','count']
    year=pd.Series([y, y, y, y, y, y, y, y, y, y]).T
    new['year']=year
    return new

In [25]:
top_drug_year=pd.concat([top_drug(y) for y in range(2008, 2018)], ignore_index=True)

In [26]:
#Check the change of top drugs during 2007 to 2017
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(top_drug_year).mark_bar().encode(
    x='count',
    y='drugName'
).transform_filter(
    click
)

hist = alt.Chart(top_condition_year).mark_point().encode(
    y='year:N',
    color=alt.condition(click, 'year:N', alt.value('lightgray'), legend=None)
).add_selection(
    click
)

scatter | hist

alt.HConcatChart(...)

<span style='color:blue'>One drug can be used for multiple conditions. Between 2008 and 2017, the most reviewed drugs switched from pain killer as the top reviewed drugs to birth control pills as dominant ones.</span>

#### 3.2 Check drugName within the same condition

In [27]:
condition_top_10=df_new.condition.value_counts().head(10)
condition_top_10

Birth Control      9648
Depression         3095
Pain               2100
Anxiety            1908
Acne               1847
Bipolar Disorde    1380
Weight Loss        1248
Insomnia           1231
Obesity            1189
ADHD               1126
Name: condition, dtype: int64

In [28]:
condition_top_10_drug=df_new[df_new.condition.isin(condition_top_10.index)][['drugName', 'condition']]
condition_top_10_drug.groupby(['condition','drugName']).sum().head(10)

Empty DataFrame
Columns: []
Index: [(ADHD, Adderall), (ADHD, Adderall XR), (ADHD, Adzenys XR-ODT), (ADHD, Amphetamine), (ADHD, Amphetamine / dextroamphetamine), (ADHD, Aptensio XR), (ADHD, Armodafinil), (ADHD, Atomoxetine), (ADHD, Budeprion XL), (ADHD, Bupropion)]

<span style='color:blue'>Similarly, one condition can be treated by different drugs. From the customers point of view, we choose drug based on conditions. </span>

### 4. rating

  **Qs:**
How is the rating distributed in this dataset across date? Which rating have the most reviews? Do drugs treated certain condition have highest rating? Is high rating revews necessarily associated with high usefulCounts? 

In [29]:
rating_dist=round(df_new.rating.value_counts(),2)
rating=rating_dist.to_frame().reset_index()
rating.columns=['rating', 'count']
rating

,rating,count
0,10.0,16937
1,9.0,9120
2,1.0,7265
3,8.0,6118
4,7.0,3075
5,5.0,2691
6,2.0,2324
7,3.0,2197
8,6.0,2102
9,4.0,1642


In [30]:
rating_dist_re=round(df_new.rating.value_counts(normalize=True),2)
rating_re=rating_dist_re.to_frame().reset_index()
rating_re.columns=['rating', 'relative_count']
rating_re

,rating,relative_count
0,10.0,0.32
1,9.0,0.17
2,1.0,0.14
3,8.0,0.11
4,7.0,0.06
5,5.0,0.05
6,2.0,0.04
7,3.0,0.04
8,6.0,0.04
9,4.0,0.03


<span style='color:blue'>Totally, the reviews with rating >= 9 are primary in this dataset, which account for ~50% of the original data set.  </span>

In [31]:
#check condition distribution across rating
def top_condition(y):
    r=df_new[df_new.rating==y].condition.value_counts().head(10)
    new=r.to_frame().reset_index()
    new.columns=['condition','count']
    rating=pd.Series([y,y,y,y,y,y,y,y,y,y]).T
    new['rating']=rating
    return new

In [32]:
top_condition(1)

,condition,count,rating
0,Birth Control,1493,1
1,Vaginal Yeast Infection,379,1
2,Depression,356,1
3,Insomnia,240,1
4,Abnormal Uterine Bleeding,235,1
5,Pain,212,1
6,Anxiety,183,1
7,Acne,179,1
8,Bipolar Disorde,178,1
9,High Blood Pressure,126,1


In [33]:
top_condition_rating=pd.concat([top_condition(y) for y in range(1, 11)], ignore_index=True)

In [34]:
#Check the change of top conditions across rating
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(top_condition_rating).mark_bar().encode(
    x='count',
    y='condition'
).transform_filter(
    click
)

hist = alt.Chart(top_condition_rating).mark_point().encode(
    y='rating:N',
    color=alt.condition(click, 'rating:N', alt.value('lightgray'), legend=None)
).add_selection(
    click
)

scatter | hist

alt.HConcatChart(...)

<span style='color:blue'> birth control and depression are the top 2 condition regardless of the rating levels.</span>

In [35]:
#Let's check rating change with date
def rating_year(y):
    r=df_new[df_new.year==y].rating.value_counts().head(10)
    new=r.to_frame().reset_index()
    new.columns=['rating','count']
    year=pd.Series([y,y,y,y,y,y,y,y,y,y]).T
    new['year']=year
    return new

In [36]:
rating_year(2008)

,rating,count,year
0,10.0,739,2008
1,9.0,429,2008
2,8.0,266,2008
3,7.0,94,2008
4,6.0,33,2008
5,5.0,24,2008
6,1.0,15,2008
7,4.0,12,2008
8,2.0,9,2008
9,3.0,8,2008


In [37]:
rating_count_year=pd.concat([rating_year(y) for y in range(2008, 2018)], ignore_index=True)
rating_count_year.head(20)

,rating,count,year
0,10.0,739,2008
1,9.0,429,2008
2,8.0,266,2008
3,7.0,94,2008
4,6.0,33,2008
5,5.0,24,2008
6,1.0,15,2008
7,4.0,12,2008
8,2.0,9,2008
9,3.0,8,2008


In [38]:
#Check the change of rating across year
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(rating_count_year).mark_bar().encode(
    x='rating',
    y='count'
).transform_filter(
    click
)

hist = alt.Chart(rating_count_year).mark_point().encode(
    y='year:N',
    color=alt.condition(click, 'year:N', alt.value('lightgray'), legend=None)
).add_selection(
    click
)

scatter | hist

alt.HConcatChart(...)

<span style='color:blue'> The interesting thing is reviews with low ratings becomes the primary contributor as time goes one. Although there was very limited low-rating reviews in 2008, there were a lot in 2018. Are people becoming picky or the drug become less effective? </span>

### 5. usefulCount

In [39]:
usefulCount_dist=df_new.usefulCount.value_counts().to_frame().reset_index()
usefulCount_dist.columns=['usefulCount', 'counts']
alt.Chart(usefulCount_dist).mark_point().encode(
    x='usefulCount:Q',
    y='counts:Q',
    tooltip=('usefulCount', 'counts')
).interactive()

alt.Chart(...)

<span style='color:blue'>Most reviews have low usefulCounts less than 100. The highest usefulCounts is 730 and the lowest is 3 </span>

In [40]:
def usefulCount_rating(y):
    r=df_new[df_new.rating==y].usefulCount.value_counts()
    new=r.to_frame().reset_index()
    new.columns=['usefulCount','count']
    rating=pd.Series([y]*len(new)).T
    new['rating']=rating
    return new

In [41]:
usefulCount_count_rating=pd.concat([usefulCount_rating(y) for y in range(1, 11)], ignore_index=True)
usefulCount_count_rating.head(20)

,usefulCount,count,rating
0,2,471,1
1,3,419,1
2,4,401,1
3,5,401,1
4,1,394,1
5,0,380,1
6,6,339,1
7,7,306,1
8,8,268,1
9,10,264,1


In [42]:
#Check the change of top conditions across rating
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(usefulCount_count_rating).mark_point().encode(
    x='usefulCount',
    y='count',
    tooltip=('usefulCount', 'count')
).transform_filter(
    click
)

hist = alt.Chart(usefulCount_count_rating).mark_point().encode(
    y='rating:N',
    color=alt.condition(click, 'rating:N', alt.value('lightgray'), legend=None)
).add_selection(
    click
)

scatter.interactive() | hist

alt.HConcatChart(...)

<span style='color:blue'>Using this interactive plot, we can see reviews with high rating doesn't necessary have high usefulCount. Rating is given by the reviewer who used the drug. usefulCounts are determined by other reviewers except the one who write the review. A good resonable review should be a review with high usefulCount.</span>

### 6. review

We will explore EDA analysis for 'review' column individually in section 2.

In [43]:
df_new.to_csv('data/df_new.csv')